# Part 1: Get Sentiment Using an LLM

You are given the daily market news of AAPL, MSFT, and TSLA from the first six months of 2023 in market_news.csv. Using the Groq API and market_news.csv, feed the news to the model and ask it to give you sentiment for each asset based on the news. 

Hint: sentiment can be broken down into negative, positive, or neutral. How can you translate this to trading signals in the next part?

## Imports
(feel free to import other things if you feel the need)

In [ ]:
%pip install pandas groq python-dotenv
import pandas as pd
from groq import Groq
import json
import os
from dotenv import load_dotenv
import time

## Configure API

Make sure to create a Groq API Key and store it in your .env file as GROQ_API_KEY.

You can create as many free API keys as you need, so if you run out of credits just make a new one

In [ ]:
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
client = Groq(api_key=GROQ_API_KEY)
model = 'llama-3.1-8b-instant'

## Use the API to Ask the Model to Give Sentiment Scores Based on the News

**Important**: to avoid rate limiting issues and to speed up the process, you should batch multiple dates/news summaries together when invoking the API call and use `time.sleep()` for per-minute rate limits.

In [ ]:

# SENTIMENTS ARE PROVIDED IN MARKET_NEWS.CSV
def get_sentiment(text):

    prompt = """You are a professional financial sentiment analyst. Given each of the sentiment summaries, classify said sentiment on a 
    sliding range between positive (1.0), neutral (0.0), or negative (-1.0). Round to the nearest tenth.

    You will be given a set of input strings in the format [IDX]: [SUMMARY].

    Return ONLY valid JSON in the following format:
    [{"row_id":IDX,"sentiment":0.8},
    {"row_id":IDX,"sentiment":-0.6}]

    Preserve the exact IDX values. Do not reorder or invent IDs.
    
    Summaries:    
    """ + text
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )
    return chat_completion.choices[0].message.content

## Get and Save the Sentiment Scores for Backtest Later

Can save them to a new csv file or any other way you can think of

In [45]:
start = 0
end = len(mndf)
# end = len(mndf)
BATCH_SIZE = 40 # Feel free to change this
mndf = pd.read_csv("market_news.csv")

for i in range(start, end, BATCH_SIZE):
    rows = mndf.iloc[i:(i + BATCH_SIZE)]
    sentiment_text = ""
    for row_id, row in rows.iterrows():
        sentiment_text += f"{row_id}: {row.summary}\n"

    sentiment_json = get_sentiment(sentiment_text)
    sentiment_json = sentiment_json.replace("```json", "")
    sentiment_json = sentiment_json.replace("```", "")
    if not sentiment_json:
        time.sleep(5)
        print(f"Error: Sentiment json output {sentiment_json}")
        continue
    try:
        sentiment_json = json.loads(sentiment_json)
        print(sentiment_json)
        for item in sentiment_json:
            mndf.loc[item["row_id"], "sentiment"] = round(float(item["sentiment"]), 1)
        mndf.to_csv("market_news.csv", index=False)
    except json.JSONDecodeError:
        print(f"Error: Sentiment json output {sentiment_json}")
        continue
    print("Done")